In [2]:
import pandas as pd

APP_NAME = 'pyspark_python'
MASTER = 'local[*]'
from pyspark import SparkConf
from pyspark.sql import SparkSession


conf = SparkConf().setAppName(APP_NAME)
conf = conf.setMaster(MASTER)
spark = SparkSession.builder.config(conf = conf).getOrCreate()
sc = spark.sparkContext

## **Spark tricks**

Some tricks to avoid some problems:

## **Use partitions**

By default, when we perform a shuffle Spark will output two hundred shuffle partitions. We will set this value from 1 to five in order to reduce the number of the output partitions from the shuffle from two hundred to five.

Go ahead and experiment with different values and see the number of partitions yourself. In experimenting with different values, you should see drastically different run times. Remenber that you can monitor the job progress by navigating to the Spark UI on port 4040 to see the physical and logical execution characteristics of our jobs.

In [4]:
import datetime

# load data

flightData2015 = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv("../data/2015-summary.csv") 


#time of execution for one partition
timestart= datetime.datetime.now()
spark.conf.set("spark.sql.shuffle.partitions", "1")
flightData2015.sort("count").take(2)

# Calculation of the time
timeend = datetime.datetime.now()
timedelta = round((timeend-timestart).total_seconds(), 2) 
print("Time require to run the model: " + str(timedelta) + " segundos")

Time require to run the model: 0.22 segundos


In [4]:
#time of execution for one partition
timestart= datetime.datetime.now()
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count").take(2)

# Calculation of the time
timeend = datetime.datetime.now()
timedelta = round((timeend-timestart).total_seconds(), 2) 
print("Time require to run the model: " + str(timedelta) + " segundos")

Time require to run the model: 0.18 segundos


## **Basic dataframe terms**
We define that we think are five basic verbs — select, filter, mutate, summarize, and arrange

In [5]:
data1 = {'PassengerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Name': {0: 'Owen', 1: 'Florence', 2: 'Laina', 3: 'Lily', 4: 'William'},
         'Sex': {0: 'male', 1: 'female', 2: 'female', 3: 'female', 4: 'male'},
         'Survived': {0: 0, 1: 1, 2: 1, 3: 1, 4: 0}}

data2 = {'PassengerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Age': {0: 22, 1: 38, 2: 26, 3: 35, 4: 35},
         'Fare': {0: 7.3, 1: 71.3, 2: 7.9, 3: 53.1, 4: 8.0},
         'Pclass': {0: 3, 1: 1, 2: 3, 3: 1, 4: 3}}

df1_pd = pd.DataFrame(data1, columns=data1.keys())
df2_pd = pd.DataFrame(data2, columns=data2.keys())

In [6]:
df1 = spark.createDataFrame(df1_pd)
df2 = spark.createDataFrame(df2_pd)
df1.show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
+-----------+--------+------+--------+



In [7]:
df1.printSchema()

root
 |-- PassengerId: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Survived: long (nullable = true)



## **Select**

In [8]:
cols1 = ['PassengerId', 'Name']
df1.select(cols1).show()

+-----------+--------+
|PassengerId|    Name|
+-----------+--------+
|          1|    Owen|
|          2|Florence|
|          3|   Laina|
|          4|    Lily|
|          5| William|
+-----------+--------+



## **Filter**

In [9]:
# one way
df1.filter(df1.Sex == 'female').show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
+-----------+--------+------+--------+



In [10]:
# second way: sql expression
df1.filter("Sex='female'").show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
+-----------+--------+------+--------+



In [12]:
## other way to filter strings
df1.filter(df1.Sex.contains("female")).show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
+-----------+--------+------+--------+



## **Mutate**: creating new columns

In [10]:
df2.withColumn('AgeTimesFare', df2.Age*df2.Fare).show()

+-----------+---+----+------+------------+
|PassengerId|Age|Fare|Pclass|AgeTimesFare|
+-----------+---+----+------+------------+
|          1| 22| 7.3|     3|       160.6|
|          2| 38|71.3|     1|      2709.4|
|          3| 26| 7.9|     3|       205.4|
|          4| 35|53.1|     1|      1858.5|
|          5| 35| 8.0|     3|       280.0|
+-----------+---+----+------+------------+



## **Summarize** using group by

In [11]:
gdf2 = df2.groupby('Pclass')
gdf2

In [12]:
avg_cols = ['Age', 'Fare']
gdf2.avg(*avg_cols).show()

+------+------------------+-----------------+
|Pclass|          avg(Age)|        avg(Fare)|
+------+------------------+-----------------+
|     3|27.666666666666668|7.733333333333333|
|     1|              36.5|             62.2|
+------+------------------+-----------------+



To call multiple aggregation functions at once, pass a dictionary.

In [13]:
gdf2.agg({'*': 'count', 'Age': 'avg', 'Fare':'sum'}).show()

+------+--------+------------------+---------+
|Pclass|count(1)|          avg(Age)|sum(Fare)|
+------+--------+------------------+---------+
|     3|       3|27.666666666666668|     23.2|
|     1|       2|              36.5|    124.4|
+------+--------+------------------+---------+



In [14]:
gdf2.agg({'*': 'count', 'Age': 'avg', 'Fare':'sum'})\
    .toDF('Pclass', 'counts', 'average_age', 'total_fare')\
    .show()

+------+------+------------------+----------+
|Pclass|counts|       average_age|total_fare|
+------+------+------------------+----------+
|     3|     3|27.666666666666668|      23.2|
|     1|     2|              36.5|     124.4|
+------+------+------------------+----------+



## **Sort**

In [15]:
df2.sort('Fare', ascending=False).show()

+-----------+---+----+------+
|PassengerId|Age|Fare|Pclass|
+-----------+---+----+------+
|          2| 38|71.3|     1|
|          4| 35|53.1|     1|
|          5| 35| 8.0|     3|
|          3| 26| 7.9|     3|
|          1| 22| 7.3|     3|
+-----------+---+----+------+



## **Joins and unions**

In [16]:
#join
df1.join(df2, ['PassengerId']).show()

+-----------+--------+------+--------+---+----+------+
|PassengerId|    Name|   Sex|Survived|Age|Fare|Pclass|
+-----------+--------+------+--------+---+----+------+
|          4|    Lily|female|       1| 35|53.1|     1|
|          3|   Laina|female|       1| 26| 7.9|     3|
|          2|Florence|female|       1| 38|71.3|     1|
|          5| William|  male|       0| 35| 8.0|     3|
|          1|    Owen|  male|       0| 22| 7.3|     3|
+-----------+--------+------+--------+---+----+------+



In [17]:
#Unions
#Union() returns a dataframe from the union of two dataframes
df1.union(df1).show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
+-----------+--------+------+--------+



One common symptom of performance issues caused by chained unions in a for loop is it took longer and longer to iterate through the loop. In this case, **repartition()** and **checkpoint()** may help solving this problem.

## **The spark.sql API**

Many of the operations that I showed can be accessed by writing SQL (Hive) queries in spark.sql().

In [18]:
df1.createOrReplaceTempView('df1_temp')
df2.createOrReplaceTempView('df2_temp')
#df.registerTempTable("connections")

In [19]:
query = '''
    select
        a.PassengerId,
        a.Name,
        a.Sex,
        a.Survived,
        b.Age,
        b.Fare,
        b.Pclass
    from df1_temp a
    join df2_temp b
        on a.PassengerId = b.PassengerId'''
dfj = spark.sql(query)


In [20]:
dfj.show()

+-----------+--------+------+--------+---+----+------+
|PassengerId|    Name|   Sex|Survived|Age|Fare|Pclass|
+-----------+--------+------+--------+---+----+------+
|          4|    Lily|female|       1| 35|53.1|     1|
|          3|   Laina|female|       1| 26| 7.9|     3|
|          2|Florence|female|       1| 38|71.3|     1|
|          5| William|  male|       0| 35| 8.0|     3|
|          1|    Owen|  male|       0| 22| 7.3|     3|
+-----------+--------+------+--------+---+----+------+



## **Create empty data.frames**

In [2]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
field = [StructField('cod_pais_1',StringType(), True),
        StructField('cod_entidad_1',IntegerType(), True),
         StructField('cod_id_1',IntegerType(), True),
         StructField('cod_persona_1',IntegerType(), True),
         StructField('fec_movim_1',IntegerType(), True),
         StructField('fec_month_1',IntegerType(), True),
         StructField('partition_1',IntegerType(), True)
        ]
schema = StructType(field)
#schema = cl_contratos_nomina.printSchema()
table_name = sqlContext.createDataFrame([], schema)

In [3]:
table_name.printSchema()

root
 |-- cod_pais_1: string (nullable = true)
 |-- cod_entidad_1: integer (nullable = true)
 |-- cod_id_1: integer (nullable = true)
 |-- cod_persona_1: integer (nullable = true)
 |-- fec_movim_1: integer (nullable = true)
 |-- fec_month_1: integer (nullable = true)
 |-- partition_1: integer (nullable = true)



### **change the names of the data.frames**

In [4]:
new_column_name_list= list(map(lambda x: x.replace("_1", ""), table_name.columns))
table_name = table_name.toDF(*new_column_name_list)

In [5]:
table_name.show()

+--------+-----------+------+-----------+---------+---------+---------+
|cod_pais|cod_entidad|cod_id|cod_persona|fec_movim|fec_month|partition|
+--------+-----------+------+-----------+---------+---------+---------+
+--------+-----------+------+-----------+---------+---------+---------+



## **schema with arrays**

**Fisrt way**

Sometime one of the columns is an array and we have to define those columns as arrays type.

* 1st. Define shema before to load.
* 2nd. Define columns that are arrays

In [14]:
from pyspark.sql.functions import split
from pyspark.sql.functions import col

schema = StructType([
    StructField("_c0",  StringType(), True),
     StructField("imp_sdopost",  StringType(), True)
])

trends = spark\
.read\
.schema(schema)\
.option("header", "true")\
.csv("data/trends.csv") 

trends.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- imp_sdopost: string (nullable = true)



In [15]:
from pyspark.sql.functions import split
from pyspark.sql.functions import col
trends= trends.withColumn("imp_sdopost", split(col("imp_sdopost"), ",").cast("array<long>"))

In [16]:
trends.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- imp_sdopost: array (nullable = true)
 |    |-- element: long (containsNull = true)



In [17]:
trends.show()

+---+--------------------+
|_c0|         imp_sdopost|
+---+--------------------+
|  0|[, 2915, 2912, 28...|
|  1|[, 228, 228, 228,...|
+---+--------------------+



**Second way**

In [20]:
'''
#(it does not work)
from pyspark.sql.types import StringType, StructField, StructType, BooleanType, ArrayType, IntegerType
schema = StructType([
        StructField("_c0", StringType(), True),
        StructField("imp_sdopost", ArrayType(
            StructType([
                StructField("element", StringType(), True),
            ])
        ), True)
    ])

trends_2 = spark\
.read\
.schema(schema)\
.option("header", "true")\
.csv("data/trends.csv") 

#trends_2.printSchema()
'''

'\n#(it does not work)\nfrom pyspark.sql.types import StringType, StructField, StructType, BooleanType, ArrayType, IntegerType\nschema = StructType([\n        StructField("_c0", StringType(), True),\n        StructField("imp_sdopost", ArrayType(\n            StructType([\n                StructField("element", StringType(), True),\n            ])\n        ), True)\n    ])\n\ntrends_2 = spark.read.schema(schema).option("header", "true").csv("data/trends.csv") \n\n#trends_2.printSchema()\n'

In [11]:
''' 
#code used  to filter by row
from pyspark.sql.functions import lit,row_number,col
from pyspark.sql.window import Window
cols= ['cod_persctpn', 'imp_sdopost']
trends= trends.select(cols)
#select some rows
w = Window().partitionBy(lit('a')).orderBy(lit('a'))
trends = trends.withColumn("row_num", row_number().over(w))
trends = trends.filter(col("row_num").between(1,2))
trends.select(['imp_sdopost']).toPandas().to_csv('data/trends.csv')
'''

' \n#filter by row\nfrom pyspark.sql.functions import lit,row_number,col\nfrom pyspark.sql.window import Window\ncols= [\'cod_persctpn\', \'imp_sdopost\']\ntrends= trends.select(cols)\n#select some rows\nw = Window().partitionBy(lit(\'a\')).orderBy(lit(\'a\'))\ntrends = trends.withColumn("row_num", row_number().over(w))\ntrends = trends.filter(col("row_num").between(1,2))\ntrends.select([\'imp_sdopost\']).toPandas().to_csv(\'data/trends.csv\')\n'

## **change squema after read data**

In [12]:
'''
df_rows = df_rows.select(
        df_rows.subT_DATE,
        df_rows.COUNT_USER.cast("float"), 
        df_rows.COUNT_NUM.cast("float"), 
        df_rows.SUMQ_TIME.cast("float"),
        df_rows.SUMELP_TIME.cast("float"), 
        df_rows.SUMCPU_TIME.cast("float")
    )
'''

'\ndf_rows = df_rows.select(\n        df_rows.subT_DATE,\n        df_rows.COUNT_USER.cast("float"), \n        df_rows.COUNT_NUM.cast("float"), \n        df_rows.SUMQ_TIME.cast("float"),\n        df_rows.SUMELP_TIME.cast("float"), \n        df_rows.SUMCPU_TIME.cast("float")\n    )\n'

## **Save CSV**

In [19]:
## other ways to do it
# with repartition
flightData2015.repartition(1).write.format("com.databricks.spark.csv").option("header", "true")\
   .mode("overwrite").save("data/data_csv_2") #path to folder

# with coalesce
flightData2015.coalesce(1).write.format("com.databricks.spark.csv")\
.option("header", "true").save("data/data_csv_3")

In [20]:
flightData2015.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [5]:
# better and simplest way
flightData2015.repartition(1).write.csv("data/test_2.csv")

## **Save and read Parquet format**

In [22]:
flightData2015.write.partitionBy("DEST_COUNTRY_NAME").format("parquet").save("data/flightData2015.parquet")

In [25]:
df = spark.read.load("data/flightData2015.parquet")
df.show(5)

+-------------------+-----+-----------------+
|ORIGIN_COUNTRY_NAME|count|DEST_COUNTRY_NAME|
+-------------------+-----+-----------------+
|            Romania|   15|    United States|
|            Croatia|    1|    United States|
|            Ireland|  344|    United States|
|              India|   62|    United States|
|          Singapore|    1|    United States|
+-------------------+-----+-----------------+
only showing top 5 rows



## **Tuning performance or debugging dataframes**

1. Cache a dataframe when it is used multiple times in the script.

In [23]:
df1.cache()

DataFrame[PassengerId: bigint, Name: string, Sex: string, Survived: bigint]

In [24]:
df1 = df1.cache()

In [25]:
df1.storageLevel

StorageLevel(True, True, False, True, 1)

In [26]:
df1.unpersist()
df1.storageLevel

StorageLevel(False, False, False, False, 1)

2. Checkpointing

Before we indicate that sometimes chaining too many union() cause performance problem or even out of memory errors. checkpoint() truncates the execution plan and saves the checkpointed dataframe to a temporary 
location on the disk.

2.1. It is recomended caching before checkpointing, so Spark doesn’t have to read in the dataframe from disk after it’s checkpointed.

2.2. To use checkpoint(), I need to specify the temporary file location to save the datafame to by accessing the sparkContext object from SparkSession.

In [27]:
#...
#sc = spark.sparkContext
sc.setCheckpointDir("checkpointdir") 

In [28]:
#For example, I can join df1 to itself 3 times:
df = df1.join(df1, ['PassengerId'])
df.join(df1, ['PassengerId']).explain()

== Physical Plan ==
*(8) Project [PassengerId#22L, Name#23, Sex#24, Survived#25L, Name#334, Sex#335, Survived#336L, Name#345, Sex#346, Survived#347L]
+- *(8) SortMergeJoin [PassengerId#22L], [PassengerId#344L], Inner
   :- *(5) Project [PassengerId#22L, Name#23, Sex#24, Survived#25L, Name#334, Sex#335, Survived#336L]
   :  +- *(5) SortMergeJoin [PassengerId#22L], [PassengerId#333L], Inner
   :     :- *(2) Sort [PassengerId#22L ASC NULLS FIRST], false, 0
   :     :  +- Exchange hashpartitioning(PassengerId#22L, 5)
   :     :     +- *(1) Filter isnotnull(PassengerId#22L)
   :     :        +- Scan ExistingRDD[PassengerId#22L,Name#23,Sex#24,Survived#25L]
   :     +- *(4) Sort [PassengerId#333L ASC NULLS FIRST], false, 0
   :        +- ReusedExchange [PassengerId#333L, Name#334, Sex#335, Survived#336L], Exchange hashpartitioning(PassengerId#22L, 5)
   +- *(7) Sort [PassengerId#344L ASC NULLS FIRST], false, 0
      +- ReusedExchange [PassengerId#344L, Name#345, Sex#346, Survived#347L], Excha

In [29]:
#I can also checkpoint() after the first join to truncate the plan.
df = df1.join(df1, ['PassengerId']).checkpoint()
df.join(df1, ['PassengerId']).explain()

== Physical Plan ==
*(4) Project [PassengerId#22L, Name#23, Sex#24, Survived#25L, Name#359, Sex#360, Survived#361L, Name#377, Sex#378, Survived#379L]
+- *(4) SortMergeJoin [PassengerId#22L], [PassengerId#376L], Inner
   :- *(1) Filter isnotnull(PassengerId#22L)
   :  +- Scan ExistingRDD[PassengerId#22L,Name#23,Sex#24,Survived#25L,Name#359,Sex#360,Survived#361L]
   +- *(3) Sort [PassengerId#376L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(PassengerId#376L, 5)
         +- *(2) Filter isnotnull(PassengerId#376L)
            +- Scan ExistingRDD[PassengerId#376L,Name#377,Sex#378,Survived#379L]


In [30]:
## **Partitions and repartition()**

Another common cause of performance problems for me was having too many partitions. I think the Hadoop world call this the small file problem. A rule of thumb: keep the partitions to ~128MB.

To check the number of partitions, use **.rdd.getNumPartitions()**

In [31]:
df1.rdd.getNumPartitions()

4

This dataframe, despite having only 5 rows, has 4 partitions. This is too many. I can repartition to only 1 partition.

In [32]:
df1_repartitioned = df1.repartition(1)
df1_repartitioned.rdd.getNumPartitions()

1

## References

* PySpark documentation [[Here]](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframewriter#pyspark.sql.DataFrameWriter) 
* PySpark Dataframe Basics [[https://changhsinlee.com/pyspark-dataframe-basics/]] 
